In [1]:
import os
import json
from typing import Literal

from dotenv import dotenv_values
import pandas as pd
from bs4 import BeautifulSoup
import plotly.express as px
import weaviate
import weaviate.classes as wvc

/var/folders/4l/8tynsjk14vg7p9s4qx1t_pb00000gn/T/ipykernel_35711/3149512216.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
config = dotenv_values("../.env")

In [3]:
client = weaviate.connect_to_custom(
    http_host="localhost",
    http_port=8080,
    http_secure=False,
    grpc_host="localhost",
    grpc_port=50051,
    grpc_secure=False,
    auth_credentials=weaviate.auth.AuthApiKey(
        config["AUTHENTICATION_APIKEY_ALLOWED_KEYS"]
    ),  # Set this environment variable
)

In [4]:
collections = list(client.collections.list_all().keys())

In [5]:
if "Subject" in collections:
    client.collections.delete("Subject")

if "UTS_Subject" in collections:
    client.collections.delete("UTS_Subject")

In [6]:
collections = list(client.collections.list_all().keys())
collections

['USYD_DOC2VEC_Subject',
 'USYD_GLOVE_Subject',
 'USYD_INSTRUCTOR_Subject',
 'USYD_SBERT_Subject',
 'UTS_DOC2VEC_Subject',
 'UTS_GLOVE_Subject',
 'UTS_INSTRUCTOR_Subject',
 'UTS_SBERT_Subject']

In [7]:
def read_json(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)
    return data

In [8]:
uni_major_equivalents = read_json("./data/major_rels/uni_major_equivalents.json")

In [9]:
uni_major_equivalents

{'MAJ10053': ['Computational Data Science'],
 'MAJ01156': ['Computational Data Science'],
 'MAJ02900': ['Cyber Security'],
 'MAJ03445': ['Cyber Security'],
 'MAJ03519': ['Software Development'],
 'MAJ02080': ['Software Development'],
 'MAJ02901': [],
 'MAJ02092': [],
 'Computational Data Science': ['MAJ10053', 'MAJ01156'],
 'Cyber Security': ['MAJ02900', 'MAJ03445'],
 'Software Development': ['MAJ03519', 'MAJ02080']}

In [10]:
uts_subject_to_majors = read_json("./data/major_rels/uts_subject_to_majors.json")
usyd_subject_to_majors = read_json("./data/major_rels/usyd_subject_to_majors.json")

In [11]:
modelType = Literal["SBERT", "INSTRUCTOR", "GLOVE", "DOC2VEC"]

In [12]:
def get_similar_uts_subjects(
    uts_subject_collection,
    usyd_subject_vector: dict[str, list[float]],
    num_subjects: int = 5,
):
    response = uts_subject_collection.query.near_vector(
        near_vector=usyd_subject_vector,
        limit=num_subjects,
        return_metadata=wvc.query.MetadataQuery(distance=True),
    )
    
    return [o.properties["subjectCode"] for o in response.objects]

In [13]:
def is_share_major(uts_majors: list[str], usyd_majors: list[str]):
    if len(uts_majors) == 0 and len(usyd_majors) == 0:
        # core subject / non-major subject case
        return True
  
    for usyd_major in usyd_majors:
        for uts_major in uts_majors:
            if usyd_major in uni_major_equivalents[uts_major]:
                return True
              
    return False

In [14]:
def is_usyd_subject_code(text: str) -> bool:
    if len(text) != 8:
        return False
      
    if text[4:].isnumeric():
        return True

In [15]:
with open("./data/usyd-comp-sci-degree.html", "r") as file:
    usyd_comp_sci_degree_html = file.read()

usyd_comp_sci_degree_soup = BeautifulSoup(usyd_comp_sci_degree_html, "html.parser")

In [16]:
current_usyd_computer_science_subjects = []

# find all strong tags
for strong_tag in usyd_comp_sci_degree_soup.find_all("strong"):
    # get the a tag in the strong tag
    a_tag = strong_tag.find("a")
    if a_tag:
        # get the text of the a tag
        text = a_tag.text
        if is_usyd_subject_code(text):
            current_usyd_computer_science_subjects.append(text)

In [17]:
def remove_non_included_subjects(uts_subject_collection, usyd_subject_collection, is_remove_usyd: bool = False):
    """
    Remove all uts subjects not in uts_subject_to_majors
    need to do this as weaviate db includes non-computer science subjects as well
    also need to remove subjects from majors that don't have equivalent majors in usyd.
    """

    non_computer_science_subjects = []
    non_equivalent_majors = [
        major
        for major in uni_major_equivalents
        if len(uni_major_equivalents[major]) == 0
    ]

    for item in uts_subject_collection.iterator():
        subject_code = item.properties["subjectCode"]

        if subject_code not in uts_subject_to_majors:
            non_computer_science_subjects.append(subject_code)
            continue

        uts_majors = uts_subject_to_majors[subject_code]

        if len(uts_majors) > 0:
            is_remove = True

            for major in uts_majors:
                if major not in non_equivalent_majors:
                    is_remove = False
                    break

            if is_remove:
                non_computer_science_subjects.append(subject_code)
    
    if len(non_computer_science_subjects) > 0:
        uts_subject_collection.data.delete_many(
            where=wvc.query.Filter.by_property("subjectCode").contains_any(
                non_computer_science_subjects
            )
        )
                
    if is_remove_usyd:
        non_current_usyd_computer_science_subjects = []
        count = 0
        
        for item in usyd_subject_collection.iterator():
            subject_code = item.properties["subjectCode"]
            
            if subject_code not in current_usyd_computer_science_subjects:
                non_current_usyd_computer_science_subjects.append(subject_code)
                count += 1
                
        # print(f"Removed {count} non-current usyd computer science subjects")
                
        if len(non_current_usyd_computer_science_subjects) > 0:
            usyd_subject_collection.data.delete_many(
                where=wvc.query.Filter.by_property("subjectCode").contains_any(
                    non_current_usyd_computer_science_subjects
                )
            )

In [18]:
def get_model_accuracy(num_similar_subjects):
    model_accuracy = {}
    share_major_vals = {}

    for model in ["SBERT", "INSTRUCTOR", "GLOVE", "DOC2VEC"]:
        num_subjects = 0
        num_share_major = 0
        num_share_major_per_subject = []

        usyd_subject_collection = client.collections.get(f"USYD_{model}_Subject")
        uts_subject_collection = client.collections.get(f"UTS_{model}_Subject")

        remove_non_included_subjects(uts_subject_collection, usyd_subject_collection, is_remove_usyd=True)

        for item in usyd_subject_collection.iterator(include_vector=True):
            subject_code = item.properties["subjectCode"]
            subject_vector = item.vector

            if type(subject_vector) != list:
                subject_vector = subject_vector["default"]

            usyd_majors = usyd_subject_to_majors[subject_code]

            similar_uts_subjects = get_similar_uts_subjects(
                uts_subject_collection, subject_vector, num_similar_subjects
            )
            
            current_num_share_major = 0

            for uts_subject_code in similar_uts_subjects:
                uts_majors = uts_subject_to_majors[uts_subject_code]

                if is_share_major(uts_majors, usyd_majors):
                    current_num_share_major += 1

                num_subjects += 1
                
            num_share_major_per_subject.append(current_num_share_major)
            num_share_major += current_num_share_major

        model_accuracy[model] = num_share_major / num_subjects
        share_major_vals[model] = num_share_major_per_subject
        
    return model_accuracy, share_major_vals

In [19]:
major_model_accuracy_top_2, _ = get_model_accuracy(2)
major_model_accuracy_top_5, histogram_data_top_5 = get_model_accuracy(5)
major_model_accuracy_top_10, histogram_data_top_10 = get_model_accuracy(10)
major_model_accuracy_top_20, histogram_data_top_20 = get_model_accuracy(20)

In [20]:
fig = px.histogram(
    x=histogram_data_top_5["INSTRUCTOR"],
    nbins=5,
    title="Number of UTS subjects that share major with USYD subjects (Top 5)",
    labels={"x": "Number of UTS subjects that share major with USYD subjects"},
)
fig.show()

In [21]:
fig = px.histogram(
    x=histogram_data_top_10["INSTRUCTOR"],
    nbins=10,
    title="Number of UTS subjects that share major with USYD subjects (Top 10)",
    labels={"x": "Number of UTS subjects that share major with USYD subjects"},
)
fig.show()

In [22]:
max(histogram_data_top_20)

'SBERT'

In [23]:
fig = px.histogram(
    x=histogram_data_top_20["INSTRUCTOR"],
    nbins=20,
    title="Number of UTS subjects that share major with USYD subjects (Top 20)",
    labels={"x": "Number of UTS subjects that share major with USYD subjects"},
)
fig.show()

In [24]:
# convert major accuracy to percentage with 4 decimal places
major_model_accuracy_top_2 = {k: round(v * 100, 2) for k, v in major_model_accuracy_top_2.items()}
major_model_accuracy_top_5 = {k: round(v * 100, 2) for k, v in major_model_accuracy_top_5.items()}
major_model_accuracy_top_10 = {k: round(v * 100, 2) for k, v in major_model_accuracy_top_10.items()}
major_model_accuracy_top_20 = {k: round(v * 100, 2) for k, v in major_model_accuracy_top_20.items()}

In [25]:
# creating a dataframe from the accuracy results
major_model_accuracy = pd.DataFrame(
    {
        "Top 2": major_model_accuracy_top_2,
        "Top 5": major_model_accuracy_top_5,
        "Top 10": major_model_accuracy_top_10,
        "Top 20": major_model_accuracy_top_20,
    }
)
major_model_accuracy

,Top 2,Top 5,Top 10,Top 20
SBERT,36.45,34.95,29.44,26.87
INSTRUCTOR,39.25,37.01,34.39,32.62
GLOVE,34.58,32.15,29.07,25.65
DOC2VEC,24.77,23.55,23.74,24.58
